<a href="https://colab.research.google.com/github/tanoManzo/mimic_trajectories/blob/dev_tano/MIMIC_TRAJECTORY_NICU_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIMIC III v1.4 dataset, Extract NICU notes with CGID not None
extract attitudes, for 50 cgid, 5 notes cg-pt, saved 'NICU_50cg_pt5notes.csv' and 'NICU_pt5notes.csv' in Data/Trajectory




#### load notes dataframe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

PATH_NOTES = "/content/drive/MyDrive/NIH/Data/NOTEEVENTS.csv"
df_notes = pd.read_csv(PATH_NOTES)
df_notes['CGID'] = df_notes['CGID'].astype('Int64')
df_notes.sample(5)

<ipython-input-2-660dc60c3942>:5: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_notes = pd.read_csv(PATH_NOTES)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
751239,755771,6995,139891.0,2192-03-09,2192-03-09 12:28:00,NaN,Radiology,P BABYGRAM (ABD ANY SGL VIEW) (74000) PORT,<NA>,NaN,[**2192-3-9**] 12:28 PM\n BABYGRAM (ABD ANY SG...
451485,456305,58917,105519.0,2148-06-14,2148-06-14 15:41:00,2148-06-14 15:59:22,Nutrition,Clinical Nutrition Note,15523,NaN,Subjective\n Patient intubated/sedated\n O...
1685998,1690482,851,164210.0,2114-04-11,2114-04-11 11:28:00,2114-04-11 11:31:00,Nursing/other,Report,20898,NaN,"Neonatology note\n6 d.o\nin RA, no issue\nwt= ..."
1365762,1319759,4436,122921.0,2147-10-09,2147-10-09 11:29:00,2147-10-09 11:46:00,Nursing/other,Report,16018,NaN,CCU Nursing Transfer Note:\n\n***Please see ad...
910118,922592,9044,191061.0,2133-08-27,2133-08-27 08:37:00,NaN,Radiology,CHEST (PORTABLE AP),<NA>,NaN,[**2133-8-27**] 8:37 AM\n CHEST (PORTABLE AP) ...


### neonatal notes

In [ ]:
file_name = "/content/drive/MyDrive/NIH/Data/ICUSTAYS.csv"
df_icu = pd.read_csv(file_name)
df_icu

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
...,...,...,...,...,...,...,...,...,...,...,...,...
61527,59806,94944,143774,201233,metavision,CSRU,CSRU,15,15,2104-04-15 10:18:16,2104-04-17 14:51:00,2.1894
61528,59807,94950,123750,283653,metavision,CCU,CCU,7,7,2155-12-08 05:33:16,2155-12-10 17:24:58,2.4942
61529,59808,94953,196881,241585,metavision,SICU,SICU,57,57,2160-03-03 16:09:11,2160-03-04 14:22:33,0.9259
61530,59809,94954,118475,202802,metavision,CSRU,CSRU,15,15,2183-03-25 09:53:10,2183-03-27 17:55:03,2.3346


In [ ]:
def subject_ids_tsicu(x):
  condition = (x['FIRST_CAREUNIT']=='TSICU') & (x['LAST_CAREUNIT']=='TSICU')
  ids = x[condition]['SUBJECT_ID'].unique()
  return ids

# get nicu notes
df_notes_tsicu = df_notes[df_notes['SUBJECT_ID'].isin(subject_ids_tsicu(df_icu))]
df_notes_tsicu

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2162-3-3**] D...
7,181,42130,114236.0,2150-03-01,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2150-2-25**] ...
14,225,11369,121504.0,2199-02-14,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2199-2-8**] Discharge ...
15,226,11369,103786.0,2203-11-12,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2203-11-3**] ...
19,230,9805,188132.0,2130-02-09,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2130-2-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
2066634,646807,53821,191606.0,2172-11-12,2172-11-12 07:59:00,2172-11-12 07:59:06,Physician,Physician Resident Progress Note,17866,NaN,Chief Complaint:\n 24 Hour Events:\n - UO ...
2066635,646809,53821,191606.0,2172-11-12,2172-11-12 07:59:00,2172-11-12 08:00:53,Physician,Physician Resident Progress Note,17866,NaN,Chief Complaint:\n 24 Hour Events:\n - UO ...
2066655,649757,78531,141755.0,2119-11-15,2119-11-15 06:30:00,2119-11-15 09:47:53,Physician,Physician Resident Progress Note,17550,NaN,Chief Complaint: change in mental status\n 2...
2066677,701673,72678,134826.0,2169-09-26,2169-09-26 07:14:00,2169-09-26 07:14:26,Physician,Physician Resident Progress Note,16654,NaN,TITLE:\n Chief Complaint:\n 24 Hour Events...


In [ ]:
a = ['422273',
'469504',
'622599',
'425699',
'725483',
'1565183',
'360026',
'378730',
'35364',
'1387881',
'1404485',
'1364610',
'729902',
'660492',
'1639164',
'1600116',
'1585435',
'1593967',
'1543544',
'1538724',
'1547271',
'1549567',
'1519785',
'1528534',
'1498693',
'1476287',
'1476284',
'1483271',
'1452615',
'1463062',
'1465889',
'1452031',
'1433466',
'1464518',
'1407462',
'1477746',
'1479061',
'1274795',
'1274024',
'1274777',
'1264096',
'1285655',
'1285918',
'1288774',
'1307826',
'1340450',
'1345973',
'1345919',
'1373236',
'1360212']

print(len(a),len(set(a)))


50 50


In [ ]:
df_notes_tsicu[df_notes_tsicu['TEXT'].apply(lambda x: ' sad ' in x.lower())]

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2162-3-3**] D...
1420,1706,77614,140792.0,2153-11-15,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2153-9-15**] ...
7533,6668,30790,134565.0,2195-10-22,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2195-10-17**] ...
13869,16126,29035,117343.0,2152-08-19,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2152-8-9**] D...
22625,31407,32349,191895.0,2186-08-07,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2186-7-12**] ...
...,...,...,...,...,...,...,...,...,...,...,...
1627020,1602920,26134,116232.0,2137-09-20,2137-09-20 14:49:00,2137-09-20 15:14:00,Nursing/other,Report,20485,NaN,"P-MICU NURSING NOTE\nS-""I'm so tired of all th..."
1635093,1648381,29892,147678.0,2113-09-21,2113-09-21 19:03:00,2113-09-21 19:30:00,Nursing/other,Report,14435,NaN,Nursing Progress Note\nSee Carevue for specifi...
1641722,1643690,29467,194819.0,2148-08-19,2148-08-19 05:58:00,2148-08-19 06:24:00,Nursing/other,Report,17704,NaN,T/SICU Nursing Progress Note\nS:\nO: Review of...
1655036,1652884,30340,100563.0,2145-02-01,2145-02-01 06:23:00,2145-02-01 06:48:00,Nursing/other,Report,19611,NaN,focus hemodymicc\ndata: neuro: alert and orien...


In [ ]:
def subject_ids_nicu(x):
  condition = (x['FIRST_CAREUNIT']=='NICU') & (x['LAST_CAREUNIT']=='NICU')
  ids = x[condition]['SUBJECT_ID'].unique()
  return ids

# get nicu notes
df_notes_nicu = df_notes[df_notes['SUBJECT_ID'].isin(subject_ids_nicu(df_icu))]
df_notes_nicu

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
6170,12240,20939,113947.0,2196-04-11,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2196-2-5**] Discharge ...
6173,12243,10201,147479.0,2148-02-05,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2148-2-3**] Discharg...
6174,12244,8026,116678.0,2140-03-21,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2140-3-19**] Discharge...
6183,12253,19967,146232.0,2168-03-08,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2168-3-2**] Discharg...
6184,12254,14712,188201.0,2110-02-04,NaN,NaN,Discharge summary,Report,<NA>,NaN,Admission Date: [**2110-1-29**] Dischar...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


## pre-processing


In [ ]:
df_notes_nicu['CATEGORY'].value_counts()

Nursing/other        404425
Radiology             14882
Discharge summary      4248
Echo                   1204
ECG                     637
Name: CATEGORY, dtype: int64

remove cgid nan

In [ ]:
df_notes_nicu_with_cgid = df_notes_nicu[~df_notes_nicu['CGID'].isna()].reset_index(drop=True)
df_notes_nicu_with_cgid['CATEGORY'].value_counts()

Nursing/other    404425
Name: CATEGORY, dtype: int64

## Exploration

In [ ]:
def get_notes_min_num_per_cg_pt(df_to_infer, num=1, opt = 0):
  """
  Fuction to remove/get cg-pt notes with a minimum or egual number

  Parameters
    ----------
    df_to_infer: dataframe
      contains the columns 'CGID', 'SUBJECT_ID' and 'TEXT'
    num: int
      minimum or egual number of notes required for cg-pt pair
    opt: bool
      0 is for minimum of num, 1 for egual to num

  Returns
    -------
    Dataframe with the required minimum/egual number of notes for cg-pt pair

  Examples
    -------
    At least 2 notes for cg-pt pair:
    >>> get_notes_min_num_per_cg_pt(df_notes_nicu_with_cgid,1,0)

    Only gc-pt with 5 notes:
    >>> get_notes_min_num_per_cg_pt(df_notes_nicu_with_cgid,5,1)
  """
  group_cg_pt = df_to_infer.groupby(['CGID', 'SUBJECT_ID'])
  if opt:
    note_condition  = group_cg_pt['TEXT'].transform('count') == num
  else:
    note_condition  = group_cg_pt['TEXT'].transform('count') > num
  return df_notes_nicu_with_cgid[note_condition].reset_index(drop=True)

In [ ]:
# at least 2 notes cg patient
df_notes_nicu_with_cgid_min_note = get_notes_min_num_per_cg_pt(df_notes_nicu_with_cgid)
info_cg_pt = df_notes_nicu_with_cgid_min_note.groupby(['CGID', 'SUBJECT_ID'])['TEXT'].count().describe()

# cg number
n_cg = df_notes_nicu_with_cgid_min_note['CGID'].nunique()
n_pt = df_notes_nicu_with_cgid_min_note['SUBJECT_ID'].nunique()
n_notes = len(df_notes_nicu_with_cgid_min_note)
print(f"number of: cg={n_cg}, pt={n_pt}, av_notes_cg_per_pt={info_cg_pt['mean']:.2f}, notes={n_notes}")

number of: cg=372, pt=4508, av_notes_cg_per_pt=4.69, notes=331545


### select notes

In [ ]:
df_notes_selected = get_notes_min_num_per_cg_pt(df_notes_nicu_with_cgid,round(info_cg_pt['mean']),0)
cg_ids = df_notes_selected.CGID.unique()
df_notes_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177332 entries, 0 to 177331
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ROW_ID       177332 non-null  int64  
 1   SUBJECT_ID   177332 non-null  int64  
 2   HADM_ID      177331 non-null  float64
 3   CHARTDATE    177332 non-null  object 
 4   CHARTTIME    177332 non-null  object 
 5   STORETIME    177332 non-null  object 
 6   CATEGORY     177332 non-null  object 
 7   DESCRIPTION  177332 non-null  object 
 8   CGID         177332 non-null  Int64  
 9   ISERROR      0 non-null       float64
 10  TEXT         177332 non-null  object 
dtypes: Int64(1), float64(2), int64(2), object(6)
memory usage: 15.1+ MB


### save dataframe

whole cgs at least 5 notes cg-pt (av. # notes)

In [ ]:
path_to_save = '/content/drive/MyDrive/NIH/Data/trajectory/'
name_to_save = 'NICU_pt_al_5notes.csv'
df_notes_selected.to_csv(path_to_save+name_to_save,index=False)

50 cg 5 notes cg-pt (av. # notes)

## Attitude


libraries

In [ ]:
! pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
selection_bound = 11
selection_cgid = cg_ids[selection_bound:selection_bound+1]

In [ ]:
condition = df_notes_selected['CGID'].isin(selection_cgid)
path_to_save = '/content/drive/MyDrive/NIH/Data/trajectory/'
name_to_save = f'NICU_cg_pt_al_5notes_0{selection_bound}.csv'
df_notes_selected[condition].to_csv(path_to_save+name_to_save,index=False)
df_notes_selected[condition].CGID.unique()

<IntegerArray>
[17293]
Length: 1, dtype: Int64

In [ ]:
len(df_notes_selected[condition])

391

models

In [ ]:
# model name from huggingface.co/model name_id:model_name
models_name = {
  'roberta':'tanoManzo/roberta-attitude',
#  'distilbert':'tanoManzo/distilbert-attitude',
#  'minilm':'tanoManzo/minilm-attitude',
#  'bloom':'tanoManzo/bloom-attitude'
}

# load tokenizer and model
tokenizers = [AutoTokenizer.from_pretrained(model_name) for model_name in models_name.values()]
models= [AutoModelForSequenceClassification.from_pretrained(model_name) for model_name in models_name.values()]

get labels and score

In [ ]:
def create_sentiment_labels(df_to_infer, batch_size=1):

  # parameters to set
  total_notes = len(df_to_infer)
  iterations = int(len(df_to_infer)/batch_size)

  # start label creation
  l_scores = {k:[] for k in models_name.keys()} # dict Model: labels
  l_labels = {k:[] for k in models_name.keys()} # dict Model: scoress

  for iter in range(iterations):
    idx_start = batch_size*iter     # first window sentence idx in the dataframe
    idx_end = batch_size*(iter+1)   # last window sentence idx in the dataframe

    # get row sentences for batch
    list_of_sentences = list(df_to_infer['TEXT'].iloc[idx_start:idx_end].values)


    for (index, (key, value)) in enumerate(models_name.items()):
      # tokenized input for the model
      tokenizer = tokenizers[index]
      batch = tokenizer(list_of_sentences, # list of sentence
                      padding=True, # add if short
                      truncation=True, # remove if long
                      max_length=512, # sent length
                      return_tensors="pt" # to return pytorch tensor (NO for TF)
                      )


      model = models[index]
      with torch.no_grad():
        outputs = model(**batch)
        predictions = F.softmax(outputs.logits, dim=1)
        label_ids = torch.argmax(predictions, dim=1)

        labels = [model.config.id2label[label_id] for label_id in label_ids.tolist()]
        scores = [round(float(predictions[idx][label_id].item()),4) for idx, label_id in enumerate(label_ids.tolist())]
        l_labels[key].extend(labels)
        l_scores[key].extend(scores)

  return l_labels, l_scores

In [ ]:
df_notes_selected.CGID.nunique()

306

In [ ]:
import pickle

for item in range(300,307):
  selection_bound = item
  selection_cgid = cg_ids[selection_bound:selection_bound+1]

  condition = df_notes_selected['CGID'].isin(selection_cgid)
  path_to_save = '/content/drive/MyDrive/NIH/Data/trajectory/'
  name_to_save = f'NICU_cg_pt_al_5notes_0{selection_bound}.csv'
  df_notes_selected[condition].to_csv(path_to_save+name_to_save,index=False)
  df_notes_selected[condition].CGID.unique()

  condition = df_notes_selected['CGID'].isin(selection_cgid)
  l_labels, l_scores = create_sentiment_labels(df_notes_selected[condition],1)
  path_to_save = "/content/drive/MyDrive/NIH/tmp results/"
  # save dictionary to person_data.pkl file
  with open(path_to_save+f'l_labels_{selection_bound}.pkl', 'wb') as fp:
      pickle.dump(l_labels, fp)
      print('dictionary saved successfully to file')

  # save dictionary to person_data.pkl file
  with open(path_to_save+f'l_scores_{selection_bound}.pkl', 'wb') as fp:
      pickle.dump(l_scores, fp)
      print('dictionary saved successfully to file')

dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file


In [ ]:

path_to_save = "/content/drive/MyDrive/NIH/tmp results/"
# save dictionary to person_data.pkl file
with open(path_to_save+f'l_labels_{selection_bound}.pkl', 'wb') as fp:
    pickle.dump(l_labels, fp)
    print('dictionary saved successfully to file')

# save dictionary to person_data.pkl file
with open(path_to_save+f'l_scores_{selection_bound}.pkl', 'wb') as fp:
    pickle.dump(l_scores, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file
dictionary saved successfully to file


In [ ]:
import pickle

path_to_save = "/content/drive/MyDrive/NIH/tmp results/"
# Read dictionary pkl file
with open(path_to_save+f'l_labels_{selection_bound}.pkl', 'rb') as fp:
    person = pickle.load(fp)
    print('Person dictionary')
    print(person)

len(person['roberta'])


Person dictionary
{'roberta': []}


0

In [ ]:
len(df_notes_selected[condition])

0

In [ ]:
print('done')

done



## Appendix

#### load patients dataframe

In [ ]:
PATH_PATIENTS = "/content/drive/MyDrive/NIH/Data/PATIENTS.csv"
df_patients = pd.read_csv(PATH_PATIENTS)
df_patients.sample(5)

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
28771,29026,30883,F,1900-08-25 00:00:00,2203-08-26 00:00:00,NaN,2203-08-26 00:00:00,1
44970,39070,71211,F,2116-06-09 00:00:00,NaN,NaN,NaN,0
4867,5703,6023,M,2066-09-30 00:00:00,2143-09-27 00:00:00,NaN,2143-09-27 00:00:00,1
44271,40019,75034,M,2090-04-20 00:00:00,2166-05-09 00:00:00,2166-05-09 00:00:00,NaN,1
41499,42084,82919,M,2080-08-02 00:00:00,NaN,NaN,NaN,0


#### load caregivers dataframe

In [ ]:
PATH_CG = "/content/drive/MyDrive/NIH/Data/CAREGIVERS.csv"
df_cg = pd.read_csv(PATH_CG)
df_cg.sample(5)

,ROW_ID,CGID,LABEL,DESCRIPTION
931,398,14329,RRT,NaN
1183,1976,15976,MD,NaN
4114,6076,20029,RO,Read Only
941,408,14339,RPh,NaN
4530,3630,17631,RO,Read Only


In [ ]:
cg_ids = df_notes_nicu_with_cgid_min_note['CGID'].unique()
df_cg[df_cg['CGID'].isin(cg_ids)]['LABEL'].value_counts()

RN        159
MD         59
PCA        32
RRT        21
MSIV       13
LICSW      10
RNC         9
SNP         8
NP          7
NNP         6
SN          6
StNuIV      4
CoWker      4
SNNP        3
md          2
HMSIV       2
MS          1
MS V        1
PT          1
Co-Wkr      1
PracSt      1
RT          1
RD/LDN      1
CoWkr       1
nnp         1
MedSt       1
HMS IV      1
StNur       1
SWInt       1
OTR/L       1
MSWint      1
SW Int      1
MS,RD       1
NSV         1
CCRN        1
RD,LDN      1
CRS         1
rn          1
StNRS       1
MSV         1
Name: LABEL, dtype: int64